# Facebook Prophet

## Setup and Import

As always, the first step is to import the required libraries and data. Since we do not want to run the SQL query every time, we can simply import the csv file we created in the first notebook.

In [23]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import altair as alt
import numpy as np

from ipywidgets import HTML
from io import BytesIO
import base64

import warnings
warnings.simplefilter("ignore")

# Turn off the max column width so the images won't be truncated
pd.set_option('display.max_colwidth', None)
# Show all Columns
pd.set_option('display.max_columns', None)
pd. set_option('display.max_rows', None)
 
# Turning off the max column will display all the data
# if gathering into sets / array we might want to restrict to a few items
pd.set_option('display.max_seq_items', 50)
pd.set_option('display.width', 1000)


In [24]:
# Import dataframes for Stock Prices
df = pd.read_csv('../data/out.csv', parse_dates=['Date'])

In [26]:
print(df['SecuritiesCode'].unique())

[1301 1332 1333 ... 9993 9994 9997]


In [27]:
#only select one Stock for testing
Code = 9993
cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'SecuritiesCode', 'Target']
STOCK = df[df.SecuritiesCode==Code][cols].set_index("Date")

test_size= 30
train_df = STOCK[:-test_size].reset_index()
test_df = STOCK[-test_size:].reset_index()

In [28]:
STOCK.head()

,Open,High,Low,Close,Volume,SecuritiesCode,Target
Date,,,,,,,
2017-01-04,1756.0,1784.0,1756.0,1773.0,5300.0,9993,0.015042
2017-01-05,1778.0,1800.0,1778.0,1795.0,6400.0,9993,0.025796
2017-01-06,1800.0,1825.0,1800.0,1822.0,8500.0,9993,-0.016586
2017-01-10,1823.0,1869.0,1810.0,1869.0,9600.0,9993,-0.003264
2017-01-11,1849.0,1864.0,1824.0,1838.0,3800.0,9993,0.000000


In [32]:
from fbprophet import Prophet
from fbprophet.make_holidays import make_holidays_df


year_list = [2017, 2018, 2019, 2020, 2021, 2022]
holidays = make_holidays_df(year_list=year_list, country='JP')
w=3

ph_df = train_df[['Date', "Close"]][w:len(STOCK)].reset_index() 
 
ph_df.rename(columns={'Close': 'y', 'Date': 'ds'}, inplace=True)

m = Prophet(growth='linear',
            changepoint_range= 0.8,
            interval_width=0.8,
            holidays=holidays,
            yearly_seasonality=False, 
            weekly_seasonality=False,
            daily_seasonality=False,
            changepoint_prior_scale=0.015)
m.add_seasonality(name = 'yearly', period=245, fourier_order=12).add_seasonality(name='monthly', period=24, fourier_order=4).add_seasonality(name='weekly', period=5, fourier_order=5)

m.fit(ph_df, verbose=0)

Initial log joint probability = -3.42591
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3509.76    0.00092473       386.044      0.7665      0.7665      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     124       3511.69   0.000550116       758.903   2.314e-06       0.001      189  LS failed, Hessian reset 
     141       3513.12   0.000104668       227.264    1.79e-07       0.001      247  LS failed, Hessian reset 
     199       3516.59   0.000358454       278.491      0.6475      0.6475      321   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     243       3517.96   9.81769e-05       347.169   3.046e-07       0.001      412  LS failed, Hessian reset 
     299       3518.97     0.0021641       207.371      0.8162      0.8162      481   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Note

In [33]:
fut = m.make_future_dataframe(periods=test_size) 
forecast = m.predict(fut)

In [34]:
from fbprophet.plot import plot_plotly, plot_components_plotly
plot_plotly(m, forecast)

## Evaluation

### Overall

In [37]:

from sklearn.metrics import mean_squared_error
y_true1 = STOCK['Close'].tail(-3)
y_true= y_true1.values
y_pred = forecast['yhat'].values

rmse = mean_squared_error(y_true, y_pred, squared=False)
mse = mean_squared_error(y_true, y_pred, squared=True)

print('MSE: %.3f' % mse)
print('RMSE: %.3f' % rmse)

MSE: 2888.744
RMSE: 53.747


In [38]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_true, y_pred)
print('Mape: %.3f' % mape)

Mape: 0.021


### Training-Data

In [39]:
from sklearn.metrics import mean_squared_error
y_true1_tr = train_df['Close'].tail(-3)
y_true_tr= y_true1_tr.values
y_pred_tr = forecast['yhat'].tail(-(test_size))
y_pred_tr = y_pred_tr.values

rmse_tr = mean_squared_error(y_true_tr, y_pred_tr, squared=False)
mse_tr = mean_squared_error(y_true_tr, y_pred_tr, squared=True)

print('MSE: %.3f' % mse_tr)
print('RMSE: %.3f' % rmse_tr)

MSE: 4065.407
RMSE: 63.761


In [40]:
from sklearn.metrics import mean_absolute_percentage_error
mape_tr = mean_absolute_percentage_error(y_true_tr, y_pred_tr)
print('Mape: %.3f' % mape_tr)

Mape: 0.024


### Test-Data

In [41]:
from sklearn.metrics import mean_squared_error
y_true1_ts = test_df['Close']
y_true_ts= y_true1_ts.values
y_pred_ts = forecast['yhat'].tail(test_size)
y_pred_ts = y_pred_ts.values

rmse_ts = mean_squared_error(y_true_ts, y_pred_ts, squared=False)
mse_ts = mean_squared_error(y_true_ts, y_pred_ts, squared=True)

print('MSE: %.3f' % mse_ts)
print('RMSE: %.3f' % rmse_ts)

MSE: 3025.277
RMSE: 55.003


In [42]:
from sklearn.metrics import mean_absolute_percentage_error
mape_ts = mean_absolute_percentage_error(y_true_ts, y_pred_ts)
print('Mape: %.3f' % mape_ts)

Mape: 0.030
